# DataFrames

Similar to pandas dataframe and SQL table.

Using RDDs in PySpark occurs a possibly large overhead between Python and the JVM.

Using DataFrames, PySpark is often significantly fast.

```python
df = spark.read.json("examples/src/main/resources/people.json")

type(df)
<class 'pyspark.sql.dataframe.DataFrame'>

df.show()
+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+

type(df['age'])     # or use df.age
<class 'pyspark.sql.column.Column'>

type(df.select('age'))
<class 'pyspark.sql.dataframe.DataFrame'>
```

## spark.createDataFrame()

```python
spark.createDataFrame([ (1,2) ], ['x', 'y']).show()
+---+---+
|  x|  y|
+---+---+
|  1|  2|
+---+---+

spark.createDataFrame([ (1,), (2,) ], ['x']).show()
+---+
|  x|
+---+
|  1|
|  2|
+---+

from pyspark.sql import Row
spark.createDataFrame([Row(a=1, intlist=[1,2,3], mapfield={"a": "b"})])
+---+---------+--------+
|  a|  intlist|mapfield|
+---+---------+--------+
|  1|[1, 2, 3]|[a -> b]|
+---+---------+--------+

from pyspark.sql.types import StructField, StructType, DoubleType
data = np.random.rand(10).round(2)
schema = StructType([StructField('value', DoubleType(), False)])
spark.createDataFrame([(float(x),) for x in data], schema=schema).show(n=3)
+-----+
|value|
+-----+
| 0.33|
| 0.07|
| 0.54|
+-----+


schema = StructType().add("id", "integer").add("country", "string")
df = spark.createDataFrame([(5, "USA"), (21, "South Korea")], schema=schema)


df = spark.createDataFrame(
    [[1, "a string", ("a nested string",)]],
    "long_col long, string_col string, struct_col struct<col1:string>")
df.show()
+--------+----------+-----------------+
|long_col|string_col|       struct_col|
+--------+----------+-----------------+
|       1|  a string|[a nested string]|
+--------+----------+-----------------+


spark.createDataFrame(pd.DataFrame([[1,2],[3,4]], columns=['x','y'])).show()
+---+---+
|  x|  y|
+---+---+
|  1|  2|
|  3|  4|
+---+---+
```

## Methods

### show(), printSchema()

show(n=20, truncate=True, vertical=False)

* n: number of rows to show
* truncate=True truncates strings longer than 20 chars by default.
* truncate=num truncates long strings to num

```python
df.show()
+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+
```

printSchema() prints out the schema in the tree format.

```python
df.printSchema()
root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)
    

df = spark.read.load("examples/src/main/resources/users.parquet")
df.show()
+------+--------------+----------------+
|  name|favorite_color|favorite_numbers|
+------+--------------+----------------+
|Alyssa|          null|  [3, 9, 15, 20]|
|   Ben|           red|              []|
+------+--------------+----------------+

df.printSchema()
root
 |-- name: string (nullable = true)
 |-- favorite_color: string (nullable = true)
 |-- favorite_numbers: array (nullable = true)
 |    |-- element: integer (containsNull = true)
```




### collect(), take(), limit()

collect() and take() returns a list of Row objects.

limit() returns a DataFrame.

```python
df.collect()
[Row(age=None, name='Michael'), Row(age=30, name='Andy'), Row(age=19, name='Justin')]

df.take(2)
[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

df.limit(2).show()
+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
+----+-------+
```


### distinct(), count(), first()

distinct() returns a new DataFrame containing the distinct rows

### corr(), cov(), approxQuantile(), crosstab()

* df.corr() or df.stat.corr()
* df.cov() or df.stat.cov()
* df.approxQuantile() or df.stat.approxQuantile()
* df.crosstab() or df.stat.crosstab()

```python
df.corr('col1', 'col2')
```

approxQuantile(col, probabilities, relativeError) calculates the approximate quantiles of numerical columns.

```python
df.approxQuantile('income', [0.25,0.5,0.75], 0.05)     # returns [Q1, median, Q3]
```

crosstab(col1, col2) computes a pair-wise frequency table of the given columns. Also known as a contingency table.

```python
df = spark.createDataFrame([('a',1),('a',2),('b',2),('b',2),('b',1),('a',1)], ['col1','col2'])
df.crosstab('col1','col2').show()
+---------+---+---+
|col1_col2|  1|  2|
+---------+---+---+
|        b|  1|  2|
|        a|  2|  1|
+---------+---+---+
```

### describe()

describe(*cols) computes basic statistics for numeric and string columns.

```python
df.describe(['col1', 'col2']).show()
```

### select(), selectExpr()

select(*cols) projects a set of expressions and returns a new DataFrame.

```python
df.select('age').collect()
[Row(age=None), Row(age=30), Row(age=19)]

df.select((df.age+10).alias('future_age')).collect()
[Row(future_age=None), Row(future_age=40), Row(future_age=29)]


df.show()
+---+---+---+---+
| id|  x|  y|  z|
+---+---+---+---+
|  1|0.4|1.2|2.4|
|  2|1.4|0.8|1.6|
|  3|0.7|2.2|0.9|
+---+---+---+---+

df.select(*['id']+[((df[c] > 2.0) | (df[c] < 0.5)).alias(c+'_outliers') for c in ['x','y','z']]).show()
# or use df.select('id', *[...]).show()
+---+----------+----------+----------+
| id|x_outliers|y_outliers|z_outliers|
+---+----------+----------+----------+
|  1|      true|     false|      true|
|  2|     false|     false|     false|
|  3|     false|      true|     false|
+---+----------+----------+----------+
```

selectExpr() is a variant of select() that accepts SQL expressions.

```python
df.selectExpr("age * 2", "abs(age)")

df.selectExpr("age", "double(height)/double(weight) as ratio")
```

### filter(), where()

where() is an alias for filter().

```python
textFile.filter(textFile.value.contains("apple"))   
# textFile is created by pyspark.read.text(). So its column name is value.


df.select(df.age, df.name).filter(df.age > 16).show()     # same as filter("age > 16")
+---+------+
|age|  name|
+---+------+
| 30|  Andy|
| 19|Justin|
+---+------+

df.filter("name like 'A%'").show()
+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+


df.filter("col1 = 'apple' AND col2 > 0").groupBy("col3", "col4").avg("col5")
```

### replace(), na.replace()

replace(to_replace, value, subset=None)


### withColumn()

withColumn(colName, col) returns a new DataFrame by adding a column or replacing the existing column that has the same name.

```python
df.withColumn('age2', df.age + 2).collect()
[Row(age=2, name='Alice', age2=4), Row(age=5, name='Bob', age2=7)]


df.withColumn('new_col', F.lit('Good'))
```

### sampleBy()

sampleBy(col, fractions, seed=None)

fractions: sampling fraction for each stratum. If a stratum is not specified, we treat its fraction as zero.

```python
df.sampleBy("key", fractions={0: 0.1, 1: 0.2}, seed=0)
```

### groupBy(), groupby(), agg(), pivot()

groupby is an alias for groupBy.

Using groupBy() with no input makes each row a single group. By doing so, we can use aggregate functions. The following are all same:

```python
df.groupBy().max("amount")
df.agg({"amount": "max"})
df.agg(F.max(df.amount))
```

df.agg() is a shorthand for df.groupBy().agg().

```python
df.agg({"age": "max", "weight": "skewness"}).collect()


from pyspark.sql import functions as F

df.agg(F.max(df.age))
# or
df.agg(F.max('age'))


df.show()
+-----+------+
|first|second|
+-----+------+
|  2.0|   4.2|
|  3.5|   2.8|
+-----+------+

df.agg(*[( (F.max(c) - F.min(c))/F.stddev(c) ).alias(c+'_transformed') for c in df.columns]).show()
+------------------+------------------+
| first_transformed|second_transformed|
+------------------+------------------+
|1.4142135623730951| 1.414213562373095|
+------------------+------------------+
```

pivot(pivot_col, values=None) pivots a column of the current DataFrame and perform the specified aggregation.

```python
# Compute the sum of earnings for each year by course with each course as a separate column

df.groupBy("year").pivot("course", ["dotNET", "Java"]).sum("earnings").collect()
[Row(year=2012, dotNET=15000, Java=20000), Row(year=2013, dotNET=48000, Java=30000)]
```

In the above, we may not specify the values of "course" in pivot(), but it is less efficient, because Spark needs to first compute the list of distinct values internally.



### join()

```python
df.join(df2, df.name == df2.name, 'outer').select(df.name, df2.height)
df.join(df2, 'name').select(df.name, df2.height)

cond = [df.name == df3.name, df.age == df3.age]
df.join(df3, cond, 'outer').select(df.name, df3.age)
df.join(df4, ['name', 'age']).select(df.name, df.age)
```

### sort(), orderBy()

```python
df.sort(df.age.desc())
df.sort("age", ascending=False)
df.sort(desc("age"))

df.orderBy(df.age.desc())
df.orderBy(desc("age"), "name")
df.orderBy(["age", "name"], ascending=[0, 1])
```    

### drop(), dropDuplicates(), drop_duplicates()

```python
df.drop('age')
df.join(df2, df.name == df2.name, 'inner').drop(df.name)


df = spark.createDataFrame([
   (1, 10.2, 'a'),
   (2, 15.8, 'b'),
   (3, 4.5, None),
   (2, 15.8, 'b'),
   (3, 10.2, 'a'),
   (1, 18.3, 'b')], ['id', 'score', 'category'])

# Distinct rows:
if df.count() != df.distinct().count(): df = df.dropDuplicates()

# Distinct (score, category):
subset = [c for c in df.columns if c != 'id']
if df.count() != df.select(subset).distinct().count(): df = df.dropDuplicates(subset)
```

### na.drop(), dropna(), na.fill(), fillna()

```mysql
df.na.fill({'age': 50, 'name': 'unknown'})
```

* Find the number of missing values in each row:

```mysql
df.rdd.map(lambda r: sum([c == None for c in r])).collect()
```


### toPandas()

df.toPandas() is a pandas dataframe.


### write.

* df.write.csv()
* df.write.json()
* df.write.parquet()
* df.write.text()
* df.write.save()
* df.write.format()

```python
df = spark.createDataFrame(...).write.csv("path/to/folder", mode='append')  # the first input is not the file name
```


## Column operations

```python
df = spark.read.json("examples/src/main/resources/people.json")
df.columns
['age', 'name']

type(df.age)
<class 'pyspark.sql.column.Column'>
```

### alias()

```python
df.select(df.age.alias('Age')).columns
['Age']
```

### cast(), astype()

```python
df.dtypes[0]
('age', 'bigint')

from pyspark.sql.types import IntegerType

df = df.withColumn('age', df.age.cast(IntegerType()))
df.dtypes[0]
('age', 'int')
```

### isNull(), isNotNull()

```python
df.filter(df.height.isNull())
```

### isin(), between()

```python
df[df.name.isin("Mike","Michael")].show()

df.select(df.name, df.age.between(2, 4)).show()
```

### startswith(), endswith()

```python
df.filter(df.name.startswith('Al')).collect()
```

### substr()

substr(startPos, length)

```python
 df.select(df.name.substr(1, 3).alias("col")).collect()
[Row(col='Mic'), Row(col='And'), Row(col='Jus')]
```

### contains()

```python
df.filter(df['value'].contains('apple')).count()
```

### when(), otherwise()

```python
df.show()
+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+

from pyspark.sql import functions as F
df.select(df.name, F.when(df.age > 25, 2).when(df.age > 15, 1).otherwise(0).alias('age_gp')).show()
+-------+------+
|   name|age_gp|
+-------+------+
|Michael|     0|
|   Andy|     2|
| Justin|     1|
+-------+------+
```

## View & SQL query

### Global temporary view

A global temporary view can be shared among all sesessions.

```python
df.createGlobalTempView("people")

# Use global_temp.name:
spark.sql("SELECT name, age FROM global_temp.people WHERE age IS NOT NULL").show()
+------+---+
|  name|age|
+------+---+
|  Andy| 30|
|Justin| 19|
+------+---+

spark.newSession().sql("SELECT name, age FROM global_temp.people WHERE age IS NOT NULL").show()
# the same result as above
```

### Temporary view

```python
df.createOrReplaceTempView("people")

spark.sql("SELECT name, age FROM people WHERE age IS NOT NULL").show()
# the same result as above

spark.newSession().sql("SELECT name, age FROM people WHERE age IS NOT NULL").show()
# Error
```

## RDD to DataFrame

### Inferring the schema using reflection

```python
rdd = sc.textFile("examples/src/main/resources/people.json")    # RDD
df = spark.read.json(rdd)                                       # DataFrame

df.printSchema()
root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)
    
    
from pyspark.sql import Row
rdd = sc.parallelize([Row(name='Michael',age=None), Row(name='Andy', age=30), Row(name='Justin', age=19)])
df = rdd.toDF()
df.show()
+-------+----+
|   name| age|
+-------+----+
|Michael|null|
|   Andy|  30|
| Justin|  19|
+-------+----+
```

### Specifying the schema

```python
from pyspark.sql.types import StructField, StructType, StringType, LongType

rdd = sc.textFile("examples/src/main/resources/people.txt") 
rdd.collect()
['Michael, 29', 'Andy, 30', 'Justin, 19']

rdd = rdd.map(lambda x: x.split(',')).map(lambda x: (x[0], int(x[1])))
rdd.collect()
[('Michael', 29), ('Andy', 30), ('Justin', 19)]

schema = StructType([
    StructField("name", StringType(), False),
    StructField("age", LongType(), True)
])

df = spark.createDataFrame(rdd, schema)
df.printSchema()
root
 |-- name: string (nullable = false)
 |-- age: long (nullable = true)
    
df.createOrReplaceTempView("people")
spark.sql("SELECT * FROM people WHERE age > 20").show()
+-------+---+
|   name|age|
+-------+---+
|Michael| 29|
|   Andy| 30|
+-------+---+
```


Another example:

```python
from pyspark.sql.types import StructField, StructType, IntegerType

rdd = sc.parallelize(['"x","y","z"', '2,3,1', '3,5,2', '8,3,4'])
header = rdd.first()
rdd = rdd.filter(lambda r: r != header).map(lambda r: r.split(',')).map(lambda r: [int(x) for x in r])
schema = StructType([StructField(c[1:-1], IntegerType(), True) for c in header.split(',')])
df = spark.createDataFrame(rdd, schema)
df.show()
+---+---+---+
|  x|  y|  z|
+---+---+---+
|  2|  3|  1|
|  3|  5|  2|
|  8|  3|  4|
+---+---+---+
```

# spark functions

## spark.range()

```python
spark.range(1,6,2).show()
+---+
| id|
+---+
|  1|
|  3|
|  5|
+---+
```

## pyspark.sql.functions

pyspark.sql.functions is a collection of built-in functions.

```python
import pyspark.sql.functions as F
```

### F.col()

F.col(name) returns a Column based on the given column name.

```python
df.select(F.col('age')*10)
# same as
df.select(df['age']*10)
```

### F.split()

```python
df = spark.createDataFrame([('oneAtwoBthreeC',)], ['s',])
df.select(F.split(df.s, '[ABC]', 2).alias('s')).show()
+-----------------+
|                s|
+-----------------+
|[one, twoBthreeC]|
+-----------------+
df.select(F.split(df.s, '[ABC]', -1).alias('s')).show()
+-------------------+
|                  s|
+-------------------+
|[one, two, three, ]|
+-------------------+
```

### F.explode()

```python
from pyspark.sql import Row

df = spark.createDataFrame([Row(a=1, intlist=[1,2,3], mapfield={"a": "b"})])

df.select(F.explode(df.intlist).alias("anInt")).show()
+-----+
|anInt|
+-----+
|    1|
|    2|
|    3|
+-----+

df.select(F.explode(df.mapfield).alias("key", "value")).show()
+---+-----+
|key|value|
+---+-----+
|  a|    b|
+---+-----+


textFile.select(F.explode(F.split(textFile.value, "\s+")).alias("word")).groupBy("word").count()
```

### F.rand(), F.randn()

rand(seed) generates a random column with i.i.d. samples uniformly distributed in [0.0, 1.0).

randn(seed) generates a column with i.i.d. samples from the standard normal distribution.

```python
df.withColumn('rand', F.rand(seed=42) * 3)
df.withColumn('randn', F.randn(seed=42) * 3)
```


### F.when().otherwise() 

```python
df.withColumn('score', F.when(F.col('score') > 80, F.col('score')).otherwise(0))
```

### F.monotonically_increasing_id()

```python
df.withColumn('unique_id', F.monotonically_increasing_id()).show()
+---+-----+--------+-------------+
| id|score|category|    unique_id|
+---+-----+--------+-------------+
|  1| 18.3|       b| 231928233984|
|  3|  4.5|    null| 231928233985|
|  1| 10.2|       a|1099511627776|
|  2| 15.8|       b|1348619730944|
+---+-----+--------+-------------+
```

### F.udf()

udf(f=None, returnType=StringType) creates a user defined function (UDF).

We create a standard Python function, wrap it with F.udf(), and use it on a DataFrame.

We call a udf with a column name of a DataFrame, and the input to the udf is a value of each row in the column.

```python
slen = F.udf(lambda s: len(s), IntegerType())  # slen.func("John") returns 4.

@F.udf               # By default, returnType is StringType().
def to_upper(s): 
    if s is not None: return s.upper()         # to_upper.func("John") returns 'JOHN'.

@F.udf(returnType=IntegerType())               
def add_one(x):                                # add_one.func(9) returns 10.
    if x is not None: return x + 1

welcome = F.udf(lambda a, b: a +', ' + b)

df = spark.createDataFrame([(1, "John Doe", 21)], ("id", "name", "age"))

df.select(slen("name").alias("name_len"), 
          to_upper("name"), 
          add_one("age").alias("age+1"), 
          welcome("name", F.lit("Good morning!")).alias("message")).show()
+--------+--------------+-----+--------------------+
|name_len|to_upper(name)|age+1|             message|
+--------+--------------+-----+--------------------+
|       8|      JOHN DOE|   22|John Doe, Good mo...|
+--------+--------------+-----+--------------------+



@F.udf(returnType=StringType())
def grade(score):
    if score < 80:
        return 'C'
    elif score < 90:
        return 'B'
    elif score <= 100:
        return 'A'
    else:
        return 'D'
df.select(grade(df['score'].cast(IntegerType())))     # if df['score'] is not integer
```

The user-defined functions are considered deterministic by default. If your function is not deterministic, call `asNondeterministic` on the user defined function.

```python
from pyspark.sql.types import IntegerType
import random
random_udf = F.udf(lambda: int(random.random() * 100), IntegerType()).asNondeterministic()

type(random_udf())
<class 'pyspark.sql.column.Column'>

random_udf.func()   # 52
random_udf.func()   # 7
```

### F.pandas_udf()

A Python UDF is executed row by row, but pandas UDFs allow vectorized operations that can increase performance.

```python
df = spark.createDataFrame(
    [[1, "a string", ("a nested string",)]],
    "long_col long, string_col string, struct_col struct<col1:string>")
df.show()
+--------+----------+-----------------+
|long_col|string_col|       struct_col|
+--------+----------+-----------------+
|       1|  a string|[a nested string]|
+--------+----------+-----------------+


@F.pandas_udf("col1 string, col2 long")
def func(s1: pd.Series, s2: pd.Series, s3: pd.DataFrame) -> pd.DataFrame:
    s3['col2'] = s1 + s2.str.len()
    return s3

df.select(func("long_col", "string_col", "struct_col").alias('result')).show()
+--------------------+
|              result|
+--------------------+
|[a nested string, 9]|
+--------------------+


df = spark.createDataFrame(pd.DataFrame([1,2,3], columns=["x"]))
def multiply_func(a: pd.Series, b: pd.Series) -> pd.Series:
    return a * b
multiply = F.pandas_udf(multiply_func, returnType=LongType())
df.select(multiply(F.col("x"), F.col("x")).alias('result')).show()
+------+
|result|
+------+
|     1|
|     4|
|     9|
+------+
```


## pyspark.sql.Window

```python
from pyspark.sql import Window

tup = [(1, "a"), (1, "a"), (2, "a"), (2, "a"), (1, "b"), (3, "b"), (4,"b")]
df = spark.createDataFrame(tup, ["id", "category"])
window = Window.partitionBy("category").orderBy("id").rangeBetween(Window.currentRow, 1)
df.withColumn("sum", F.sum("id").over(window)).sort('category','id').show()
+---+--------+---+
| id|category|sum|
+---+--------+---+
|  1|       a|  6|    # 1+1+2+2
|  1|       a|  6|    # 1+1+2+2
|  2|       a|  4|    # 4+4
|  2|       a|  4|    # 4+4
|  1|       b|  1|    # 1
|  3|       b|  7|    # 3+4
|  4|       b|  4|    # 4
+---+--------+---+
```

Note that `orderBy("id").rangeBetween(Window.currentRow, 1)` does not mean that the range is between the current row and the next row. It means that the window is between the first row whose is same as the current row and the last row whose id is (the current id + 1). 

Use `Window.unboundedPreceding`, `Window.unboundedFollowing`, and `Window.currentRow` to specify special boundary values, rather than using integral values directly.

```python
window = Window.partitionBy("category").orderBy("id").rangeBetween(Window.unboundedPreceding, Window.currentRow)
df.withColumn("sum", F.sum("id").over(window)).sort('category','id').show()
+---+--------+---+
| id|category|sum|
+---+--------+---+
|  1|       a|  2|
|  1|       a|  2|
|  2|       a|  6|
|  2|       a|  6|
|  1|       b|  1|
|  3|       b|  4|
|  4|       b|  8|
+---+--------+---+
```

What is the difference between the revenue of each product and the revenue of the best selling product in the same category as that product? (from databricks.com)

```python
windowSpec = Window.partitionBy(df['category'])\
  .orderBy(df['revenue'].desc())\
  .rangeBetween(Window.unboundedPreceding, Window.unboundedFollowing)

revenue_difference = 
df.select('product', 
          'category', 
          'revenue', 
          (F.max('revenue').over(windowSpec) - df['revenue']).alias("revenue_difference"))
```

# Load, Write, Structured streaming


## spark.read.

* text(paths, wholetext=False, lineSep=None, ...)
* csv(...)
* parquet(...)

```python
textFile = spark.read.text("README.md")    # DataFrame with column name 'value'
```

## Structured streaming

```python
df.show()
+------+----+---------+
|  name| age|      job|
+------+----+---------+
| Jorge|  30|Developer|
|   Bob|  32|Developer|
|  John|  28| Engineer|
|Gloria|null|   Artist|
+------+----+---------+

df.printSchema()
root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- job: string (nullable = true)
    
df.write.csv("people_csv", mode="append")

sdf = spark.readStream.csv(path='people_csv', schema=df.schema)

sdf.isStreaming        # True

query = sdf.groupBy('job').count()\
  .writeStream.start(queryName='num_jobs', outputMode='complete', format='memory')

spark.sql("SELECT * FROM num_jobs").show()
+---------+-----+
|      job|count|
+---------+-----+
|Developer|    2|
|   Artist|    1|
| Engineer|    1|
+---------+-----+

df2 = spark.createDataFrame([("Pam", 29, "Engineer"), ("Tom", 35, "Marketer"), ("Ben", 33, "Marketer")], schema = df.schema)
df2.write.csv("people_csv", mode="append")

spark.sql("SELECT * FROM num_jobs").show()
+---------+-----+
|      job|count|
+---------+-----+
|Developer|    2|
|   Artist|    1|
| Engineer|    2|
| Marketer|    2|
+---------+-----+
```

We can join a stream dataframe with a static dataframe:

```python
room_info = spark.createDataFrame([('Developer', 'A302'), ('Engineer', 'A300'), ('Artist','A303'), ('Marketer', 'B250')], ['job', 'room_no'])

sdf_joined = sdf.join(room_info, 'job')
sdf_joined.isStreaming      # True

query = joined_df.writeStream.start(queryName='join_query', outputMode='append', format='memory')
spark.sql("SELECT * FROM join_query").show()
+---------+------+----+-------+
|      job|  name| age|room_no|
+---------+------+----+-------+
|Developer|   Bob|  32|   A302|
|Developer| Jorge|  30|   A302|
| Engineer|   Pam|  29|   A300|
| Engineer|  John|  28|   A300|
|   Artist|Gloria|null|   A303|
| Marketer|   Ben|  33|   B250|
| Marketer|   Tom|  35|   B250|
+---------+------+----+-------+
```